In [39]:
import cv2
import sys
import numpy as np
from collections import Counter
num_train = 2000
image_train = []
for i in range(num_train):
    image = cv2.imread("/Users/himeshagrawal/Downloads/assn2 2/train/%d.png" % i)
    image_train.append(image)

file = open( "/Users/himeshagrawal/Downloads/assn2 2/train/labels.txt", "r" )
label_train = file.read().splitlines()
file.close()
label_train = np.array(label_train)
image_train = np.array(image_train)
print(type(image_train.shape))
print(label_train)

<class 'tuple'>
['ODD' 'ODD' 'ODD' ... 'EVEN' 'ODD' 'EVEN']


In [40]:
def remove_lines(img):
  _, binary = cv2.threshold(img, 210, 255, cv2.THRESH_BINARY)
  kernels = np.ones((5,5), np.uint8)
  dilated = cv2.dilate(binary, kernels, iterations=1)
  return dilated

In [42]:
def pre_processing(image):
    cv2.imwrite(f'/Users/himeshagrawal/Downloads/assn2 2/images/final0.png',image)
    top_left = (0, 0)
    top_right = (image.shape[0]-1, 0)
    bottom_left = (0, image.shape[1]-1)
    bottom_right = (image.shape[0]-1, image.shape[1]-1)
    
    color_frequencies = {}
    for corner in [top_left, top_right, bottom_left, bottom_right]:
        color = image[corner]
        color_tup = tuple(color)
        if color_tup not in color_frequencies:
           color_frequencies[color_tup] = 0
        color_frequencies[color_tup] += 1

    max_color = max(color_frequencies, key=color_frequencies.get)
    R = max_color[0]
    G = max_color[1]
    B = max_color[2]

    h,w,c = image.shape
    for i in range(h):
        for j in range(w):
           if R == image[i,j,0] and G == image[i,j,1] and B == image[i,j,2]:
            image[i,j,0] = 255
            image[i,j,1] = 255
            image[i,j,2] = 255

    cv2.imwrite(f'/Users/himeshagrawal/Downloads/assn2 2/images/final1.png',image)
    img2 = image
    image_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    pixels = image_rgb.reshape(-1, 3)
    non_white_pixels = [pixel for pixel in pixels if not np.array_equal(pixel, [255, 255, 255])]
    color_counts = Counter(map(tuple, non_white_pixels))
    most_common_color = color_counts.most_common(1)[0][0]
    print(most_common_color)
    R = most_common_color[0]
    G = most_common_color[1]
    B = most_common_color[2]
    img2 = image_rgb
    h,w,c = img2.shape
    for i in range(h):
        for j in range(w):
            if R!= img2[i,j,0]:
                img2[i,j,0] = 255
                img2[i,j,1] = 255
                img2[i,j,2] = 255
            if G != img2[i,j,1]:
                img2[i,j,0] = 255
                img2[i,j,1] = 255
                img2[i,j,2] = 255
            if B != img2[i,j,2]:
                img2[i,j,0] = 255
                img2[i,j,1] = 255
                img2[i,j,2] = 255

    cv2.imwrite(f'/Users/himeshagrawal/Downloads/assn2 2/images/final3.png',img2)
    return img2


In [49]:
p4 = np.zeros((2000,100,100,3))
for i in range(6):
    p4[i] = pre_processing(image_train[i][:,350:450,:])
    cv2.imwrite(f'/Users/himeshagrawal/Downloads/assn2 2/images/final5.png', p4[i])
    print(i)

(255, 130, 128)
0
(140, 255, 128)
1
(159, 255, 128)
2
(128, 0, 0)
3
(255, 138, 128)
4
(128, 130, 255)
5


In [ ]:
# # Convert string labels to numerical labels
# train_labels = [0 if label == 'ODD' else 1 for label in label_train]

# train_labels = np.array(train_labels)

In [ ]:
num_ref = 16
image_ref = []
label_ref = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F']
for i in range(num_ref):
    image = cv2.imread("/Users/himeshagrawal/Downloads/assn2 2/reference/%c.png" % label_ref[i])
    image_ref.append(image)


lable_ref = np.array(label_ref)
image_ref = np.array(image_ref)

In [ ]:
image_ref.shape

(16, 100, 100, 3)

In [ ]:
num_test = 7
image_test = []
for i in range(num_test):
    image = cv2.imread("/Users/himeshagrawal/Downloads/assn2 2/dummy_submit/test/%d.png" % i)
    image_test.append(image)

file = open( "/Users/himeshagrawal/Downloads/assn2 2/dummy_submit/model.txt", "r" )
label_test = file.read().splitlines()
file.close()
label_test = np.array(label_test)
image_test = np.array(image_test)

In [ ]:
p1_test = np.zeros((7,100,125,3))
p2_test = np.zeros((7,100,125,3))
p3_test = np.zeros((7,100,125,3))
p4_test = np.zeros((7,100,125,3))
for i in range(7):
    image_test[i] = pre_processing(image_test[i])
    p1_test[i],p2_test[i],p3_test[i],p4_test[i] = segement(image_test[i])
    print(i)



0
1
2
3
4
5
6


In [ ]:
# Convert string labels to numerical labels
# test_labels = [0 if label == 'ODD' else 1 for label in label_test]
# test_labels = np.array(test_labels)

In [4]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

data_dir = '/Users/himeshagrawal/Downloads/assn2 2/seg_image'

image_size = (100, 100)
# Load the images and convert them to grayscale
images = []
for i in range(2000):
    image_path = os.path.join(data_dir, str(i) + ".png")
    image = Image.open(image_path).convert("L")
    image = np.array(image.resize(image_size)).flatten()  # Resize to specified size and flatten
    images.append(image)

# Step 2: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, label_train, test_size=0.1, random_state=42)


# # Step 4: Make predictions on the testing set
# y_pred = clf.predict(X_test)

# # Step 5: Evaluate the model
# accuracy = accuracy_score(y_test,y_pred)
# print("Accuracy:", accuracy)

In [5]:
k = 5  # Number of neighbors
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train,y_train)

# Step 4: Predict the labels for the test set
y_pred = knn.predict(X_test)

# Step 5: Evaluate the accuracy of the classifier
accuracy = classification_report(y_test, y_pred)
print(accuracy)

              precision    recall  f1-score   support

        EVEN       1.00      1.00      1.00       112
         ODD       1.00      1.00      1.00        88

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Initialize the decision tree classifier
dt = DecisionTreeClassifier()

# # Define the parameter grid for fine-tuning
# param_grid = {
#     'max_depth': [3, 5, 7],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 3, 5],
# }

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(dt, param_grid, cv=5)
# grid_search.fit(X_train, y_train)

# # Get the best model and its parameters
# best_model = grid_search.best_estimator_
# best_params = grid_search.best_params_

# Train the decision tree classifier with the best parameters
dt.fit(X_train, y_train)

# Make predictions on the test set
y_pred = dt.predict(X_test)

# Calculate the accuracy of the model
accuracy = classification_report(y_test, y_pred)
print(accuracy)

              precision    recall  f1-score   support

        EVEN       0.97      0.98      0.98       112
         ODD       0.98      0.97      0.97        88

    accuracy                           0.97       200
   macro avg       0.98      0.97      0.97       200
weighted avg       0.98      0.97      0.97       200



In [23]:
print(best_params)

{'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Initialize the logistic regression classifier
lr = LogisticRegression(solver = 'liblinear',penalty = 'l1',max_iter=100)
# # Define the parameter grid for fine-tuning
# param_grid = {
#     'C': [0.1, 1, 10],
#     'penalty': ['l1', 'l2'],
#     'solver': ['liblinear', 'saga']
# }

# # # Perform grid search with cross-validation
# grid_search = GridSearchCV(lr, param_grid, cv=2)
lr.fit(X_train, y_train)

# # # Get the best model and its parameters
# best_model = grid_search.best_estimator_
# best_params = grid_search.best_params_

# Make predictions on the test set
y_pred = lr.predict(X_test)

# Calculate the classification report
classification_rep = classification_report(y_test, y_pred)
print(classification_rep)

              precision    recall  f1-score   support

        EVEN       1.00      1.00      1.00       112
         ODD       1.00      1.00      1.00        88

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [ ]:
file = open( "/Users/himeshagrawal/Downloads/assn2 2/train/labels.txt", "r" )
label_train = file.read().splitlines()
file.close()
label_train = np.array(label_train)
image_train = np.array(image_train)
print(type(image_train.shape))
print(len(label_train))

In [10]:
import pickle

# Assuming you have a trained model called "model"
model = lr  # Your trained model object

# Define the file path to store the model
model_file = '/Users/himeshagrawal/Downloads/assn2 2/model_l.pkl'

# Save the model to a file
with open(model_file, 'wb') as file:
    pickle.dump(model, file)